In [91]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, log_loss
from torchsummary import summary
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import matplotlib.pyplot as plt

In [117]:
import pandas as pd
df = pd.read_csv('../Cases/Concrete Strength/Concrete_Data.csv')
df.head()

,Cement,Blast,Fly,Water,Superplasticizer,Coarse,Fine,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [118]:
lbl = LabelEncoder()
y = lbl.fit_transform( df['Strength'].values )
X = df.drop('Strength', axis=1).values

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, 
                                                    random_state=24)
X_scl_trn = scaler.fit_transform(X_train) 
X_scl_tst = scaler.transform(X_test) 

y_scl_trn = scaler.fit_transform(y_train.reshape(-1,1))
y_scl_tst = scaler.transform(y_test.reshape(-1,1))

# y_scl_trn = scaler.fit_transform(y_train)
# y_scl_tst = scaler.transform(y_test)


X_torch = torch.from_numpy(X_scl_trn)
y_torch = torch.from_numpy(y_scl_trn)



print(X_torch.size())
print(y_torch.size())

torch.Size([721, 8])
torch.Size([721, 1])


In [119]:
torch.manual_seed(24)
model = nn.Sequential(nn.Linear(in_features=X_scl_trn.shape[1], out_features=1))

summary(model, (1,X_scl_trn.shape[1]))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1, 1]               9
Total params: 9
Trainable params: 9
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [125]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [126]:
for p in model.parameters():
    print(p)

y_pred = model(X_torch.float())
y_pred[:3]

# y_torch = y_torch.unsqueeze(1)
print(y_torch.shape)
print(y_pred.shape)

Parameter containing:
tensor([[ 1.6211,  0.4550, -0.1528, -0.8807,  1.2560, -0.5048, -0.7010,  1.5207]],
       requires_grad=True)
Parameter containing:
tensor([-0.1048], requires_grad=True)
torch.Size([721, 1])
torch.Size([721, 1])


In [127]:
for epoch in np.arange(0,1000):
       # Forward pass: Compute predicted y by passing x to the model
       y_pred_prob = model(X_torch.float())

       # Compute and print loss
       loss = criterion(y_pred_prob, y_torch.float())
       if epoch%100 == 0:
          print('epoch: ', epoch+1,' loss: ', loss.item())

       # Zero gradients, perform a backward pass, and update the weights.
       optimizer.zero_grad()

       # perform a backward pass (backpropagation)
       loss.backward()

       # Update the parameters
       optimizer.step()
#print('epoch: ', epoch+1,' loss: ', loss.item())


epoch:  1  loss:  0.42154061794281006
epoch:  101  loss:  0.0554366260766983
epoch:  201  loss:  0.038201458752155304
epoch:  301  loss:  0.034611910581588745
epoch:  401  loss:  0.033512502908706665
epoch:  501  loss:  0.03307865932583809
epoch:  601  loss:  0.03287103399634361
epoch:  701  loss:  0.032754164189100266
epoch:  801  loss:  0.03267953172326088
epoch:  901  loss:  0.03262759745121002


In [128]:
X_torch_test = torch.from_numpy(X_scl_tst)
### Inferencing on test set
lin_output = model(X_torch_test.float()) # Equivalent predict_proba / predict

np_out = lin_output.detach().numpy()
# y_pred_prob = 1 / (1 + np.exp(-np_out))


# y_pred_prob = y_pred_prob.reshape(y_test.shape[0],) 

# y_pred = np.where(y_pred_prob >= 0.5,1,0)
y_pred = lin_output.detach().numpy()

In [129]:
from sklearn.metrics import r2_score
r2_score(y_scl_tst, y_pred)

0.5820568203681511